# Выбор локации для скважины

Были предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Построим модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируем возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

In [2]:
class PredictionData(object):
    __slots__ = (        
        'features_train', 
        'target_train', 
        'features_valid', 
        'target_valid',
    )

## Загрузка и подготовка данных

In [3]:
import pandas as pd
import numpy as np
import scipy.stats as st

from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [4]:
df_0 = pd.read_csv("/datasets/geo_data_0.csv")

df_1 = pd.read_csv("/datasets/geo_data_1.csv")

df_2 = pd.read_csv("/datasets/geo_data_2.csv")


display(df_0.shape)
display(df_0.head(20)) 

print("Пропущенных значений: ", df_0.isna().sum().sum())

print("Дубликатов:", df_0.duplicated().sum())

(100000, 5)

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
5,wX4Hy,0.969570,0.489775,-0.735383,64.741541
6,tL6pL,0.645075,0.530656,1.780266,49.055285
7,BYPU6,-0.400648,0.808337,-5.624670,72.943292
8,j9Oui,0.643105,-0.551583,2.372141,113.356160
9,OLuZU,2.173381,0.563698,9.441852,127.910945


Пропущенных значений:  0

Дубликатов: 0


In [5]:
display(df_1.shape)
display(df_1.head(20))

print("Пропущенных значений: ", df_1.isna().sum().sum())

print("Дубликатов:", df_1.duplicated().sum())

(100000, 5)

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
5,HHckp,-3.327590,-2.205276,3.003647,84.038886
6,h5Ujo,-11.142655,-10.133399,4.002382,110.992147
7,muH9x,4.234715,-0.001354,2.004588,53.906522
8,YiRkx,13.355129,-0.332068,4.998647,134.766305
9,jG6Gi,1.069227,-11.025667,4.997844,137.945408


Пропущенных значений:  0

Дубликатов: 0


In [6]:
display(df_2.shape)
display(df_2.head(20)) 

print("Пропущенных значений: ", df_2.isna().sum().sum())

print("Дубликатов:", df_2.duplicated().sum())

(100000, 5)

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
5,LzZXx,-0.758092,0.710691,2.585887,90.222465
6,WBHRv,-0.574891,0.317727,1.773745,45.641478
7,XO8fn,-1.906649,-2.458350,-0.177097,72.480640
8,ybmQ5,1.776292,-0.279356,3.004156,106.616832
9,OilcN,-1.214452,-0.439314,5.922514,52.954532


Пропущенных значений:  0

Дубликатов: 0


id — уникальный идентификатор скважины;

f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);

product — объём запасов в скважине (тыс. баррелей).

Вывели на экран три файла геологоразведки по регионам, ознакомились с содержимым и проверили на наличие пропусков и дубликатов. Можно приступать к обучению и проверки модели.

## Обучение и проверка модели

Создадим функцию, берущую на вход датасет и делящую его на train и test выборки, а также обучает модели.

In [7]:
def split_and_train_df(df):

    features = df.drop(["product","id"], axis=1)
    target = df["product"]
    features_train, features_valid, target_train, target_valid = train_test_split(features, 
                                                                                  target, 
                                                                                  test_size=0.25, 
                                                                                  random_state=1)
    scaler = MinMaxScaler()
    features_train = scaler.fit_transform(features_train)
    features_valid = scaler.transform(features_valid)
    
    model = LinearRegression(normalize=False)
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    predictions = pd.Series(predictions)
    rmse = (mean_squared_error(predictions, target_valid))**(0.5)
    average_product = sum(predictions) / len(predictions)
    
    print("RMSE: {0:.2f}".format(rmse))
    print("Average product: {0:.2f}".format(average_product))
    
    return (predictions, target_valid.reset_index(drop=True), rmse)

In [8]:
i=1
for df in [df_0, df_1, df_2]:
    print(f"Region {i}:")
    print("\n----")
    split_and_train_df(df)
    print("---\n\n")
    i+=1 

Region 1:



----

RMSE: 37.74

Average product: 92.49

---





Region 2:



----

RMSE: 0.89

Average product: 69.12

---





Region 3:



----

RMSE: 39.87

Average product: 94.96

---






In [9]:
# Сохраним предсказания, валидационные данные и RMSE 
predictions_0, validation_0, rmse_0 = split_and_train_df(df_0)
predictions_1, validation_1, rmse_1= split_and_train_df(df_1)
predictions_2, validation_2, rmse_2 = split_and_train_df(df_2)

RMSE: 37.74

Average product: 92.49

RMSE: 0.89

Average product: 69.12

RMSE: 39.87

Average product: 94.96


Лучше всего модель предсказала данные по второму региону (RMSE: 0.89), предсказание остальных регионов было менее удачным. Однако рассматривать второй регион в дальнейшем не будем - в нем слишком мало запасов. 

## Подготовка к расчёту прибыли

In [10]:
budget = 1e+07 # бюджет в тысячах рублей за тысячу баррелей
point_per_budget = 200 # количество разрабатываемых точек
product_price = 450 # доходность одной единицы барреля в тысячах рублей
chosen_points = 500

Расчитаем безубыточную разработку сырья на 200 месторождениях. И найдем среднее количество продукта в месторождениях необходимое для этого.

In [11]:
average_profit_per_point = budget / point_per_budget
print('Средняя прибыль на лучшую точку -', average_profit_per_point, 'тыс.руб.')

average_barrels_per_point = average_profit_per_point  / product_price
print('Средний объем нефти на точку -',average_barrels_per_point, 'тыс. барелей')

average_barrels = budget / product_price
print('Средний объем нефти на регион -',average_barrels, 'тыс. барелей')

Средняя прибыль на лучшую точку - 50000.0 тыс.руб.

Средний объем нефти на точку - 111.11111111111111 тыс. барелей

Средний объем нефти на регион - 22222.222222222223 тыс. барелей


In [12]:
print('Средний запас фактичекой нефти на скважину в регионе 1:', df_0['product'].mean())
print('Средний запас фактичекой нефти на скважину в регионе 2:', df_1['product'].mean())
print('Средний запас фактичекой нефти на скважину в регионе 3:', df_2['product'].mean())

Средний запас фактичекой нефти на скважину в регионе 1: 92.50000000000001

Средний запас фактичекой нефти на скважину в регионе 2: 68.82500000000002

Средний запас фактичекой нефти на скважину в регионе 3: 95.00000000000004


По найденным данным средних запасов и нижней точки безубыточности можно сделать вывод, что ни один регион не обладает требуемым запасом нефти в скважинах.

## Расчёт прибыли и рисков 

In [13]:
def profitization(predictions, target): 
    best_preds = predictions.sort_values(ascending=False) 
    best_target = target[best_preds.index][:200] 
    revenue = best_target.sum() * product_price 
    return revenue - budget

In [14]:
state = np.random.RandomState(12345) 
def bootstraping_1000(predictions, target): 
    values = [] 
    for i in range(1000):
        target_sample = target.sample(500, replace=True, random_state=state,) 
        predictions_sample = predictions[target_sample.index]
        values.append(profitization(predictions_sample, target_sample))
    values = pd.Series(values) 
    lower = values.quantile(0.025) 
    upper = values.quantile(0.975) 
    average = values.mean() 
    risk = st.percentileofscore(values, 0)
    return ((lower, upper), average, risk)

In [15]:
i = 1
for preds, target in zip([predictions_0, predictions_1, predictions_2], 
                         [validation_0, validation_1, validation_2]):
    interval, mean_revenue, risk = bootstraping_1000(preds, target)
    print(f"Region {i}:")
    print("\n----")
    print(f"Mean: {mean_revenue}")
    print(f"95% confidence interval: {interval}")
    print(f"Risk of loss: {risk}")
    print('   ')
    i+=1

Region 1:



----

Mean: 458100.40179975115

95% confidence interval: (-91938.45332589102, 1015824.8886825135)

Risk of loss: 4.6000000000000005

   

Region 2:



----

Mean: 533056.7096039738

95% confidence interval: (99430.44840112432, 973064.42488865)

Risk of loss: 0.6000000000000001

   

Region 3:



----

Mean: 411076.6725586822

95% confidence interval: (-138944.97961462854, 959378.9183216941)

Risk of loss: 7.2

   


<div class="alert alert-block alert-danger">
<b>Ошибка:</b>  Здесь код упал, посмотри пожалуйста
  
</div>

<div class="alert alert-block alert-warning">
    <b> Исправил имена переменных в блоке кода выше, все должно работать</b>  
</div>

Исходя из полученных данных, можно сделать вывод, что второй регион - самый благоприятный по балансу средней прибыли и рискам.

<div class="alert alert-block alert-success">
<b>Успех[2]:</b> Есть контакт
</div>



# Вывод

1) Провели исследовательский анализ данных: вывели на экран три файла геологоразведки по регионам, ознакомились с содержимым и проверили на наличие пропусков и дубликатов.

2) Обучили модель для каждого региона: df_0, df_1, df_2

3) Затем применили bootstrap и нашли 95% доверительный интервал для среднего объема нефти в скважинах и риск убытков. Наиболее перспективным для разработки представляется регион № 2, так как риски убытков наименьшие, а средняя прибыль выше, чем в остальных регионах.